# II. PARTIE WEBSCRAPPING

In [1]:
!pip install Scrapy

# Import os => Library used to easily manipulate operating systems
## More info => https://docs.python.org/3/library/os.html
import os 

# Import logging => Library used for logs manipulation 
## More info => https://docs.python.org/3/library/logging.html
import logging

# Import scrapy and scrapy.crawler 
import scrapy
from scrapy.crawler import CrawlerProcess

## Le scrapping sera fait sur 8 villes sélectionnées selon les plus hautes températures et la prévision : les 3 plus hautes températures par prévision

NOTE BUG pour la cellule suivante : scrapper booking.com par ville

1. Tentative de faire une boucle pour faire le scrapping de toutes les villes en une fois mais ça s'arrête au bout de 2 villes avec l'erreur "reactor not restartable"\
Résultat : le scrapping a été fait ville par ville

2. Problème sur le sélecteur des scores : "valeur nulle"

# II.a Scrapping via Srapy

In [2]:
ville = 'Paris'

In [3]:
class Hotel(scrapy.Spider):

    # Name of your spider
    name = "info_hotel"

    # Starting URL
    start_urls = ['https://www.booking.com/index.fr.html']

    # Parse function for login
    def parse(self, response):
        # FormRequest used to login
        return scrapy.FormRequest.from_response(
            response,
            formdata={'ss': ville},
            callback=self.after_search
        )
    
      # Callback used after login
    
    def after_search(self, response):

        for info in response.css('.sr_item'): 
            yield {
                'name': info.css('.sr-hotel__name::text').get(),
                'url': "https://www.booking.com" + info.css('.hotel_name_link').attrib["href"],
                'coords': info.css('.sr_card_address_line a').attrib["data-coords"],
                'score': info.css('review-score-badge::text').get(),
                'description': info.css('.hotel_desc::text').get()            
            }

        # Select the NEXT button and store it in next_page
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            # In the last page, there won't be any "href" and a KeyError will be raised
            logging.info('No next page. Terminating crawling process.')
        else:
            # If a next page is found, execute the parse method once again
            yield response.follow(next_page, callback=self.after_search)

# II.b Sauvegarde des informations hotel pour les villes selectionnées

In [4]:

filename = "hotels_infos_" + ville + ".json"

if filename in os.listdir('src/'):
        os.remove('src/' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename: {"format": "json"},
    }
})

process.crawl(Hotel)
process.start()

2021-01-17 22:47:11 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-01-17 22:47:11 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-4.19.112+-x86_64-with-glibc2.10
2021-01-17 22:47:11 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-01-17 22:47:11 [scrapy.extensions.telnet] INFO: Telnet Password: 72ea3f9c8742c88e
2021-01-17 22:47:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-01-17 22:47:11 [scrapy.middleware] INFO: Ena